# **Notebook Information**

The following notebook has been setup to give you quick access to the next round of games so you can run model.predict against. Some suggestions/information for using this notebook:<br>
<br>
- The following works only for the standard columns from the Footy_Tipping_starter.ipynb, if you created additional features that you used for training you will need to add them here <br>
- I would recommend saving you prediction model as a .pkl file, then use joblib to open it. 

# **Libraries**

In [ ]:
import pandas as pd

from datetime import datetime
import requests

pd.set_option('display.max_columns', None)

# Functions for data preperation
from functions.add_leading_zero import add_leading_zero
from functions.get_season_stats import get_season_stats
from functions.get_table_hist import get_table_hist
from functions.clean_table_data import clean_table_data
from functions.clean_games_data import clean_games_data
from functions.get_next_round import get_next_round
from functions.get_next_round_bk import get_next_round_bk

from requests.packages.urllib3.util.retry import Retry

# **Load Data**

In [ ]:
# Team Code data for joins between datasets
data = {'Team': ['Adelaide', 'Brisbane Lions', 'Brisbane Bears', 'Carlton', 'Collingwood', 'Essendon', 'Fitzroy', 'Fremantle', 'Geelong', 'Gold Coast', 'Greater Western Sydney', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'University', 'West Coast', 'Western Bulldogs'],
        'Code': [1, 19, 2, 3, 4, 5, 6, 8, 9, 20, 21, 10, 11, 12, 13, 14, 15, 16, 17, 18, 7],
        'Abv': ['AD', 'BL', 'BB', 'CA', 'CW', 'ES', 'FI', 'FR', 'GE', 'GC', 'GW', 'HW', 'ME', 'NM', 'PA', 'RI', 'SK', 'SY', 'UN', 'WC', 'WB']}

team_code = pd.DataFrame(data)
team_code['Code'] = team_code['Code'].apply(add_leading_zero)

The below code will get some game and tabe data from the previous season which will be joined to the current round.<br>

When the season begins you will need to adjust the end year to 2023

In [ ]:
# Load the season data
games = get_season_stats(start_year=2022, end_year=2023)

# Load the 2022 Table data
table_hist = get_table_hist(start_year=2022, end_year=2023)

In [ ]:
try:
    round_df = get_next_round()
except KeyError:
    round_df = get_next_round_bk()

In [ ]:
# Create a dataframe that concats the games hist and the next round
new_df = pd.concat([games, round_df], sort=False)

In [ ]:
# Create a copy of the table
table_hist_2 = table_hist.copy()

# Clean the table data
table_hist_2 = clean_table_data(table_hist_2)

# Clean the games data and then join with the new_df
games_clean = clean_games_data(new_df, team_code, table_hist_2)

# Drop the target column
games_clean = games_clean.drop('target', axis=1)

# **Select the Current Round for Prediction**

In [ ]:
# Select the current round
current_round = games_clean.loc[games_clean.isna().any(axis=1)]

# Reset the index
current_round.reset_index(inplace=True, drop=True)


In [ ]:
current_round